## Introducción

En el siguiente proyecto trabajaré con la base de datos de una compañía móvil en la cual analizaré el comportamiento de sus clientes con el objetivo de recomendar un nuevo plan. Para realizar estas recomendaciones, desarrollaré un modelo de clasificación que escoja correctamente cual de los planes se ajusta mejor a cada usuario de acuerdo con sus características, con la mayor exactitud posible, estableciendo un umbral de 0.75. 

#### Secciones a desarrollar:

1. Inicialización: importar las librerías con las que trabajaré y cargar los datos en un DataFrame.
2. Segmentar los datos para el modelo de aprendizaje.
3. Comparar la calidad de diferentes modelos de clasificación.
4. Elegir un modelo y comprobar su calidad con el conjunto de prueba.
5. Prueba de cordura en el modelo elegido.
6. Conclusiones generales.

## 1. Incialización

In [1]:
#importar librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
#librerías para elegir un modelo
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## 1.1 Cargar datos

In [3]:
#cargar dataset
users_behav = pd.read_csv("datasets/users_behavior.csv")

In [4]:
#visualizar informacion general del dataset
users_behav.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#visualizar datos
users_behav.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [6]:
#revisar valores duplicados en el dataframe
users_behav.duplicated().sum()

0

In [7]:
#revisar valores ausentes en el dataframe
users_behav.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [8]:
#visualizar estadísticas generales en el dataframe
users_behav.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Al realizar una exploración inicial en la base de datos, encuentro que se encuentra en buen estado para comenzar a trabajar con ella. No se encontraron filas duplicadas ni valores ausentes y, en cuanto a los tipos de datos en las columnas, estos están correctos.

## 2. Segmentar los datos para el modelo

In [9]:
#definir variables para las características y el objeto del modelo
features = users_behav.drop(['is_ultra'], axis=1)
target = users_behav['is_ultra']

#segmentar un conjunto de entranamiento y uno de validación 
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.20, random_state=12345)

#segmentar el conjunto de validacion en uno de testeo y otro de validación
features_test, features_valid_test, target_test, target_valid_test = train_test_split(features_valid, target_valid, test_size=0.20, random_state=12345)

He segmentado el dataset fuente en tres conjuntos: set de entrenamiento, set de validación y set de testeo, con los que pondré a prueba cada uno de los modelos de clasificación. Posteriormente, elegiré un modelo definitivo para implementar con el dataset fuente completo.

## 3. Comparar la calidad de los modelos

### 3.1 Árbol de decisión de regresión

In [10]:
#árbol de decisión de regresión
best_model = None
best_result = 10000
best_depth = 0

for depth in range (1,6):
    model = DecisionTreeRegressor(random_state=12345, max_depth=depth, min_samples_split=2, min_samples_leaf=1, splitter='best')
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)

    result = mean_squared_error(target_valid, prediction_valid) ** 0.5

    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth


print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")
print()
print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")
print(f"Exactitud del modelo en el conjunto de testeo = {model.score(features_test, target_test)}")

RECM del mejor modelo en el conjunto de validación (max_depth = 5): 0.4015534785850737

Exactitud del modelo en el conjunto de validación = 0.239069247263892
Exactitud del modelo en el conjunto de entrenamiento = 0.331617391235569
Exactitud del modelo en el conjunto de testeo = 0.25464931595726614


Al probar un árbol de decisión de regresión me doy cuenta de que este modelo no alcanza el umbral de exactitud de 0.75 requerido para este proyecto, alcanzando 0.33. Y parece estar sobreajustado. Descarto este modelo por que el nivel de exactitud.

### 3.2 Bosque aleatorio de regresión

In [11]:
#RandomForest regresor
best_model = None
best_result = 10000
best_depth = 0
best_est = 0

for est in range(10, 50, 100):
    for depth in range(1, 11):
        model = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth, min_samples_leaf=1, min_samples_split=2, max_features=1)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        
        result = mean_squared_error(target_valid, prediction_valid) ** 0.5

        if result < best_result:
            best_model = model
            best_result = result
            best_depth = depth
            best_est = est


print(f"RECM del mejor modelo en el conjunto de validación= {best_result}, n_estimators= {best_est}, best_depth= {best_depth}")
print()
print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")
print(f"Exactitud del modelo en el conjunto de testeo = {model.score(features_test, target_test)}")

RECM del mejor modelo en el conjunto de validación= 0.4015534785850737, n_estimators= 10, best_depth= 1

Exactitud del modelo en el conjunto de validación = 0.285532574581872
Exactitud del modelo en el conjunto de entrenamiento = 0.5279879923905082
Exactitud del modelo en el conjunto de testeo = 0.3237165591283928


Un bosque aleatorio de 10  árboles, con profundidad de 1 da un nivel de exactitud de 0.52 en el conjunto de entrenamiento y parece estar sobreajustado, por lo tanto, no resulta suficiente para la tarea.

### 3.3 Regresión logística

In [9]:
#regresión logística
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=500, penalty='l2')
model.fit(features_train, target_train)

predict_valid = model.predict(features_valid)

results = mean_squared_error(target_valid, predict_valid)

print(f"RECM del mejor modelo en el conjunto de validación = {results}")
print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")

RECM del mejor modelo en el conjunto de validación = 0.24261275272161742
Exactitud del modelo en el conjunto de validación = 0.7573872472783826
Exactitud del modelo en el conjunto de entrenamiento = 0.7467911318553092


La regresión logística tiene un error cuadrático menor en comparación con el bosque aleatorio y una exatitud de 0.75 en el conjunto  de validación y 0.74 en el conjunto de entrenamiento. No da indicio de estar sobreajustado. Por lo tanto, evaluaré la elección final de este modelo.

### 3.4 Regresión lineal

In [10]:
#regresión linear
model = LinearRegression()
model.fit(features_train, target_train)

predicts_valid = model.predict(features_valid)

result = mean_squared_error(target_valid, predicts_valid)

print(f"RECM del mejor modelo en el conjunto de validación = {result}")
print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")

RECM del mejor modelo en el conjunto de validación = 0.19642912305724222
Exactitud del modelo en el conjunto de validación = 0.07303309479416342
Exactitud del modelo en el conjunto de entrenamiento = 0.08995572905826121


La regresión lineal proporciona la menor exactitud de todos los modelos probados, por lo tanto queda descartado.

## 4. Comprobar la calidad del modelo usando el conjunto de prueba

El modelo elegido para esta tarea será una regresión logística, el cual utilizaré con el conjunto de prueba. Elegí este modelo porque me dió la más alta exactitud en el conjunto de validación (0.75) y al compararlo con el conjunto de entrenamiento, no parece estar sobreajustado.

In [11]:
#comprobar la calidad del LogisticRegression en el conjunto de prueba
features
target

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=500, penalty='l2')
model.fit(features_train, target_train)

test_predictions = model.predict(features)

mse = mean_squared_error(target, test_predictions)

print(f"Exactitud del modelo en el conjunto de prueba = {model.score(features, target)}")
print(f"RECM del mejor modelo en el conjunto de prueba = {mse}")


Exactitud del modelo en el conjunto de prueba = 0.7489110143123833
RECM del mejor modelo en el conjunto de prueba = 0.2510889856876167


En el conjunto de prueba, la regresión logística proporciona una exactitud del 74%, bastante parecido a la manera en que se comportó con el conjunto de validación. 

## 5. Prueba de cordura

In [12]:
#segmentar los datos para la prueba
features_sanity = users_behav.drop(['calls', 'is_ultra'], axis=1)
target_sanity = users_behav['calls']

print('Número promedio de llamadas realizadas:', target_sanity.mean())

predict_sanity = pd.Series(target_sanity.mean(), index=target_sanity.index)

mse_sanity = mean_squared_error(target_sanity, predict_sanity) ** 0.5

print('RECM de la prueba de cordura:', mse_sanity)
print('Número máximo de llamadas realizadas:', users_behav['calls'].max())

Número promedio de llamadas realizadas: 63.03889234598631
RECM de la prueba de cordura: 33.23119658396647
Número máximo de llamadas realizadas: 244.0


Para realizar la prueba de cordura, utilicé el valor promedio de la cantidad de llamadas realizadas por los clientes y calculé el RECM utilizando ese valor. 

La RECM indica que, en promedio, las predicciones erraron por aproximadamente 33 llamadas, siendo que en el conjunto, el número de llamadas de algunos clientes llega hasta 244.

## Conclusión general

Para cumplir con el objetivo de este proyecto probé varios modelos de regresión cambiando sus hiper-parámetros para ver cual de todos me proporccionaba el nivel más alto de exactitud, sin sobreajustarse. 

El modelo elegido fue una regresión logística, el cual, mediante comprobar la calidad, me proporcionó un nivel de exatitud de 0.74 en el conjunto de prueba. 
Adicionalmente se realizó una prueba de cordura, en la que separé las llamadas realizadas por los clientes y calculé su valor promedio para utilizarlo como valor predictivo al calcular posteriormente el error cuadrático medio. El resultado indicó que el modelo falló por aproximadamente 33 llamadas, teniendo en cuenta que algunos clientes realizan hasta 244 llamadas mensualmente.

Como conclusión yo diría que el modelo funciona bastante bien en el conjunto de datos final y que al menos en el 74% de las veces, seremos capaces de recomenar el plan correcto a cada cliente.